In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
import sklearn.metrics as metrics
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df = pd.read_excel("BQ Analysis v1_1.xlsx", "Project BQ")

In [3]:
df = df.astype(str)
df["PREPROCESS_TEXT"] = df["BQ_TRADE"] + "," + df["BQ_HEADING"] + "," + df["BQ_SUB_HEADING"] + "," + df["BQ_ITEM_HEADING"] + "," + df["BQ_ITEM_DESCRIPTION"] + "," + df["AC_TRADE_CODE"]
#df['RESULT'] = ""
#training_df = df.loc[df['CONTRACT_CODE'] == "FL49"].copy() // not enough training data?
training_df = df[df['CONTRACT_CODE'] != "QHP3"]
test_df = df[df['CONTRACT_CODE'] == "QHP3"]

df["PREPROCESS_TEXT"] = list(map(lambda x: x.lower(), list(df["PREPROCESS_TEXT"])))


In [4]:

training_labels = tuple(training_df["AC_TRADE_CODE"])
training_texts = tuple(training_df["PREPROCESS_TEXT"])
#print(training_labels)
#print(training_text)



#test_labels = tuple(test_df["RESULT"])
test_texts = tuple(test_df["PREPROCESS_TEXT"])


In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
#vectorizer = CountVectorizer()
X = vectorizer.fit_transform(training_texts)
y = training_labels


clf = SGDClassifier()
clf.fit(X, y)


X_test = vectorizer.transform(test_texts)
#y_test = test_labels

test_predictions = clf.predict(X_test)

annotated_test_data = list(zip(test_predictions, test_texts))

#y_test = np.array(test_labels)
print(metrics.classification_report(test_df["AC_TRADE_CODE"], test_predictions))
print("Accuracy: %0.4f" % metrics.accuracy_score(test_df["AC_TRADE_CODE"], test_predictions))

              precision    recall  f1-score   support

          AP       0.00      0.00      0.00         0
          AR       0.93      1.00      0.96       215
          AS       1.00      0.62      0.77        45
          BR       1.00      1.00      1.00        35
          BS       0.00      0.00      0.00         0
          CJ       0.99      1.00      1.00       368
          DA       0.00      0.00      0.00        12
          DR       1.00      0.95      0.98       479
          EL       0.97      0.74      0.84      1426
          EX       0.22      1.00      0.36        10
          FD       0.00      0.00      0.00         0
          GL       1.00      1.00      1.00        95
          IR       1.00      1.00      1.00        24
          PA       0.95      0.99      0.97       427
          PL       0.94      1.00      0.97       853
          PR       1.00      1.00      1.00        58
          PS       0.98      1.00      0.99      2788
          SM       0.98    

C:\Users\frankielau\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\frankielau\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
test_df['RESULT'], test_df['PROCESS_TEXT'] = zip(*annotated_test_data)
test_df["COMPARE"] = np.where(test_df["AC_TRADE_CODE"] == test_df["RESULT"], True, False)
#test_df

<ipython-input-6-0b8584eff0e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['RESULT'], test_df['PROCESS_TEXT'] = zip(*annotated_test_data)
<ipython-input-6-0b8584eff0e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["COMPARE"] = np.where(test_df["AC_TRADE_CODE"] == test_df["RESULT"], True, False)


In [7]:
test_df['COMPARE'].value_counts()

True     11429
False      458
Name: COMPARE, dtype: int64

In [8]:
#Save Model
with open('text_vectorizer', 'wb') as picklefile:
    pickle.dump(vectorizer,picklefile)

with open('text_classifier', 'wb') as picklefile:
    pickle.dump(clf,picklefile)